In [1]:
import torch
from torch.nn import Linear, MSELoss
from torch.optim import Adam

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Create a simple model
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.layer1 = Linear(5, 22500)
        self.layer2 = Linear(22500, 22500)
        self.layer3 = Linear(22500, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return self.layer3(x)

# Instantiate the model
model = SimpleModel().cuda()

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# Use a basic Adam optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Sample data
data = torch.randn(20, 5).cuda()
target = torch.randn(20, 1).cuda()
loss_func = MSELoss()

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(data)
    loss = loss_func(output, target)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss {loss.item()}")


506.430001 M parameters


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.89 GiB (GPU 0; 10.92 GiB total capacity; 9.44 GiB already allocated; 897.12 MiB free; 9.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF